In [49]:
%matplotlib inline

import oauth2
from twython import Twython
import simplejson
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import requests
import datetime
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import sys

import unittest, time, re

In [3]:
#APP_KEY = "qtmevmQ18N1vyWTAXfxqmh4oN"
#APP_SECRET = "MdZibormo3teZPTfMyeLEcuzMURHYidArOml0GtOQyrl6dI13R"

#access_token = '2694571580-Y8DsMjB0iMTGmm3Pwpo6IL3enhhFdAZQSXDIxO8'
#access_secret = 'AYciwyU197r6adpNziDT8pB0tmT3bKIihMrx7SPfbofRO'

#Define Twitter GET function using OAUTH2
#Function from https://dev.twitter.com/oauth/overview/single-user
#def oauth_req(url, key, secret, http_method="GET", post_body="", http_headers=None):
#    consumer = oauth2.Consumer(key=APP_KEY, secret=APP_SECRET)
#    token = oauth2.Token(key=key, secret=secret)
#    client = oauth2.Client(consumer, token)
#    resp, content = client.request( url, method=http_method, body=post_body, headers=http_headers )
#    return content

#twitter = Twython(APP_KEY, APP_SECRET, access_token, access_secret)


In [50]:
#We borrow heavily from http://stackoverflow.com/questions/12519074/scrape-websites-with-infinite-scrolling
def scrape_page(since, until, contestant, \
                base_url="https://twitter.com/search?f=tweets&vertical=default&q=%23thebachelor", \
                pages_to_scroll=3, ):
    
    #### Initiate Chrome Browser #######
    #Must download ChromeDriver executable from https://sites.google.com/a/chromium.org/chromedriver/downloads
    driver = webdriver.Chrome('/Users/dcusworth/chrome_driver/chromedriver') #Specify location of driver
    driver.implicitly_wait(30)
    verificationErrors = []
    accept_next_alert = True

    #Create URL that will get the text
    ender = "&src=typd"
    
    #Use Twitter Sentiment Analysis - REMOVED as it may be underestimating tweets
    #if is_happy:
    #    sentiment = "%20%3A)"
    #else:
    #    sentiment = "%20%3A("
    
    since_time = "%20since%3A" + str(since)
    until_time = "%20until%3A" + str(until)
    contestant_name = "%20" + contestant    
        
    final_url = base_url + contestant_name  + since_time + until_time + ender
    #print final_url
    
    #Jump onto the webpage and scroll down
    delay = 3
    driver.get(final_url)
    for i in range(1,pages_to_scroll):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
    html_source = driver.page_source
    
    #After scrolling enough times, get the text of the page
    data = html_source.encode('utf-8')
    driver.quit()

    return data

## Week 1 Twitter

In [ ]:
contestants = ["Olivia", "Lauren B", "LB", "Caila", "Amber", "Jami", \
               "Jennifer", "Jubilee", "Amanda", "JoJo", "Leah", "Rachel", \
               "Samantha", "Jackie", "Haley", "Emily", "Shushanna", "Lauren H", \
               "Becca", "Mandi", "Lace"]

In [60]:
week1_tweets = {}

for cont in contestants:
    npage = scrape_page("2016-01-04", "2016-01-06", cont, pages_to_scroll=25)

    soup = BeautifulSoup(npage, "html.parser")
    user_tweets = soup.find_all("p", attrs={"class": "TweetTextSize"})

    each_tweet = [uu.get_text() for uu in user_tweets]
    N = len(each_tweet)
    
    week1_tweets.update({cont:N})

In [61]:
with open('week1.json', 'w') as fp:
    json.dump(week1_tweets, fp)